# visualizeKraken2 v.beta0.8

2022.10.29

Implemented as a class

In [ ]:
import os
import shutil
import ftplib
import zipfile
import re
import numpy as np
import plotly.express as px
import pandas as pd
import time

In [ ]:
version = 'beta0.8'

### Startup functions

In [ ]:
def downloadDatabase(databaseFolder, databaseFileName):
    """
    If the correct file is not in its local place, this function removes the whole database 
    folder and downloads everything from scratch from the NCBI server. The remove is done
    to avoid problems with old or incomplete files from previous downloads.    
    """
    start = time.time()
    print('\nvisualizeKraken2 v.' + version)
    print('------------------------------\n')
    print('Checking for database source file...')
    
    if not os.path.isfile(str(databaseFolder) + '/fullnamelineage.dmp'): # this is the file used to build the database
        if os.path.isfile(databaseFileName):
            os.remove(databaseFileName) # this removes any old zip file that might cause problems.
        shutil.rmtree(databaseFolder) # os module stops you because of windows admin rights.
        os.mkdir(databaseFolder)
        print('directory "' + str(databaseFolder) + '" created\n')
        with ftplib.FTP('ftp.ncbi.nlm.nih.gov') as ftp: # connecting to the NCBI server.
            print('Downloading taxonomy database from NCBI...\n')
            print('First, a scary message from the US government:')
            print(ftp.getwelcome()) # no idea what this means..
            print('\nAt least the server is ready.\n\nDownloading...\n')
            try:
                ftp.login()
                ftp.cwd('pub/taxonomy/new_taxdump/') # navigating to the correct directory in the server.
                with open(databaseFileName, "wb") as lf:
                    ftp.retrbinary("RETR " + databaseFileName, 
                                   lf.write, 8*1024) # download in binary to prevent file corruption
            
            except ftplib.all_errors as error:
                print('FTP error: ',  error)
                print("This means something has gone wrong with the server. \
                      Make sure you're connected to the internet and try again.")
            
        with zipfile.ZipFile(databaseFileName, 'r') as zipFile: # unzips file
            zipFile.extractall(databaseFolder)
        
        os.remove(databaseFileName) # removes the zip file and any downloaded files which are not needed.
        for file in os.listdir(databaseFolder):
            if file != 'fullnamelineage.dmp':
                #print(str(databaseFolder) + '/' + str(file) + ' removed') # can be used for troubleshooting.
                os.remove(str(databaseFolder) + '/' + str(file))
        print('\nDatabase downloaded.\n')
        end = time.time()
        print('Time elapsed:', str(round(end-start, 1)), 'seconds\n')

    else:
        print('\nDatabase source file exists already. No download required.\n')

In [ ]:
def buildDatabase(databaseFolder):
    """
    Builds a python dictionary from the source file downloaded from NCBI.
    This is the most time-consuming step in the program and so is done before any analysis is started.
    """
    start = time.time()
    dataBase = {}
    print('Building database...\n')
    with open(str(databaseFolder) + '/fullnamelineage.dmp', 
              errors='ignore') as inputF: # decoding the .dmp file = problematic. Errors are now ignored. Downstream problems?
        for line in inputF: # this can be done more elegantly
            tempList1 = re.split(r'\t\|\t', line)
            tempList2 = re.split(r'; ', tempList1[2])
            tempList3 = []
            tempList3.append(tempList1[1])
            tempList3.append(tempList2[:-1])
            dataBase[tempList1[0]] = tempList3
            #for key,item in dataBase.items(): # can be used for troubleshooting
                #print(key, item)
    
    print('Database built.\n')
    print('Length of dictionary "dataBase":',len(dataBase))
    end = time.time()
    print('Time to build database:', str(round(end-start, 1)), 'seconds\n')
    return dataBase

In [ ]:
databaseFolder = "downloaded_NCBI_taxonomy_database" 
# this can be changed if you prefer another folder name.

databaseFileName = "new_taxdump.zip" 
# this is the file name on the NCBI server. Don't change. If unexplainable errors occur, check if this file's 
# name has changed on the NCBI server. It is unlikely though.
# zip file most compatible with different systems? There are .gz varieties

In [ ]:
downloadDatabase(databaseFolder, databaseFileName)

In [ ]:
dataBase = buildDatabase(databaseFolder)

### Class

In [ ]:
class krakView:
    """
    Defines a class for visualizing and displaying data from an imported kraken2 txt file.
    Input file must be in the same folder as the script, unless the entire path is provided.
    """
    VERSION = "Version " + version
    
    def __init__ (self, krakenFile, filter = 5):
        """
        Creates an object from a kraken2 txt file named according to the following pattern: filename.kraken2     
        """
        self.sourceFile = krakenFile
        self.data = np.loadtxt(krakenFile, dtype=str, delimiter='\t')
        self.uniqueIDs, self.count = np.unique(self.data[:,2], return_counts=True)
        self.count = self.count[1:]
        self.uniqueIDs= self.uniqueIDs[1:] 
        
    def showFormat(self, n = 5):
        """
        Prints the n first lines of the kraken file. Default n = 5.
        """
        return self.data[0:n, :]
    
    def percentClassified(self, output = 1):
        """
        Returns percentage of sucessfully classified fasta sequences from the kraken analysis.
        output = 1 means a descriptive message will be displayed. Can be disabled by output = 0.
        """
        Us = np.sum(np.char.count(self.data[:, 0], 'U'))
        Cs = np.sum(np.char.count(self.data[:, 0], 'C'))
        percentageC = float(round((Cs/(Cs+Us))*100,1))
        if output == 1:
            print('The percentage of classified sequences in the fasta file: ' + str(percentageC) + '%')
        return percentageC
    
    def numberUniqueIDs(self, output = 1):
        """
        Returns the number of unique taxonomy IDs found in the kraken file. 
        Output = 1 displays a message, output = 0 disables it.
        This can be a guideline to what to filter by. Maybe add a range.
        """
        if output == 1:
            print('Number of unique taxIDs found: ' + str(len(self.uniqueIDs)))
        return len(self.uniqueIDs)
    
    def mapIDs(self, minMatches = 5):
        """
        Maps the IDs in the kraken file to the species names and ancestors in the NCBI database.
        
        minMatches specifies filtering by number of matches found for a specific taxon,
        for example filter = 5 maps only the taxons with more than 5 matches in the original fasta.
        Too many entries may cause problems with plotly visualization, so filtering by 5 or 10 is 
        recommended. 
        
        "root" means that NCBI could not place it more precisely than at the root of its taxonomy tree, 
        which basically tells us nothing except that the sequence contained nucleotides. These entries are removed.
        
        Commented out prints can be used for troubleshooting.   
        """
        speciesCounts = [] # number of matches for each ID
        speciesNames = [] # names of the LCAs
        speciesLineage = [] # full lineage in a list
        for i in range(len(self.uniqueIDs)):
            #print('\ni=', i, '\n')
            if self.count[i] > minMatches: # Filter by number of matches.
                #print('i=', i)
                if self.uniqueIDs[i] == '1':
                    print('"' + str(dataBase[self.uniqueIDs[i]][0]) + '" ' + 'with ' + 
                          str(self.count[i]) + ' counts removed from list.')
                else:
                    try:
                        #print(self.dataBase[self.uniqueIDs[i]][0])
                        speciesNames.append(dataBase[self.uniqueIDs[i]][0])
                        reversedAncestors = list(dataBase[self.uniqueIDs[i]][1])
                        reversedAncestors.reverse() # reversing to make parsing in ascending order easier
                        speciesLineage.append(reversedAncestors)
                        speciesCounts.append(self.count[i]) 
                    except:
                        #print('i=', i)
                        print('ID', self.uniqueIDs[i], 'not in database')
            
        if len(speciesCounts) == len(speciesNames) == len(speciesLineage):
            print(len(speciesLineage), 'unique taxons found.')
        else:
            print('Something went horribly wrong with database mapping. Visualization will probably not work :(')
        
        # This next part arranges the data in the format required by plotly to produce the visualizations.
        
        char = [] # creates a list of characters, each character is an object in the plot
        parent = [] # the closest parent of the enrty in char. Every object in parent must also be in char.
        match = [] # the number of matches for the taxon in question.

        listOfRootNodes = ['cellular organisms', 
                          'Viruses', 
                          'other entries', 
                          'unclassified entries']

        for i in range(len(speciesNames)):
            #print('\n---------- entry', i,  '--------')
            #print('Name:', speciesNames[i], '\n')
            #print('Lineage list:', speciesLineage[i])
            if speciesNames[i] in listOfRootNodes:
                char.append(speciesNames[i])
                match.append(speciesCounts[i])        
                parent.append('')
                #print('i=', i, '\nroot APPENDED as parent to', speciesNames[i], '\n')
            else:
                char.append(speciesNames[i])
                match.append(speciesCounts[i])
                parent.append(speciesLineage[i][0])
                #print('i=', i, speciesLineage[i][0], 'appended as PARENT to', speciesNames[i])
                for j in range(len(speciesLineage[i])):
                    #print('Current ancestor:', speciesLineage[i][j])
                    if speciesLineage[i][j] not in speciesNames and speciesLineage[i][j] not in char:
                        if speciesLineage[i][j] in listOfRootNodes:
                            #print('i=', i)
                            char.append(speciesLineage[i][j])
                            #print(speciesLineage[i][j], 'appended to char')
                            match.append(0)
                            parent.append('')
                            #print('root appended as parent to', speciesLineage[i][j])
                        else:
                            char.append(speciesLineage[i][j])
                            #print(speciesLineage[i][j], 'appended to char')
                            match.append(0)
                            parent.append(speciesLineage[i][j+1])
        
        if len(char) == len(parent) == len(match):
            print(len(char), 'entries in total after adding parents to all.\n')
        else:
            print('Something went horribly wrong with parent mapping. Visualization will probably not work :(')
        
        return char, parent, match
    
    def organizeData(self, char, parent, match):
        """
        Organizes the data for plotly visualization.
        """
        percentMatches = []
        total = sum(self.count) # count or match? hmm
        for i in match:
            percent = round((i/total)*100, 2)
            percentMatches.append(percent)
            
        plotData = dict(LCA = char,
                        Parent = parent,
                        PercentMatches = percentMatches)
        return plotData
    
    def showList(self, n=25, minMatches = 5):
        """
        Returns a list of the n taxons with the largest number of matches.
        Adding percentMatches later
        And maybe statistics from col 5
        """
        char, parent, match = self.mapIDs(minMatches)
        #plotData = self.organizeData(char, parent, match)
        matchesAndLca = pd.DataFrame({'Matches' : match, 'LCA' : char})
        matchesAndLcaSorted = matchesAndLca.sort_values('Matches', ascending=False)
        matchesAndLcaSortedTopResults = matchesAndLcaSorted.head(n=n)
        print('List of the top', n, 'taxons with most matches:\n')
        print(matchesAndLcaSortedTopResults.to_string(index=False))
    
    def showSunburst(self, minMatches = 5, width = 1100, height = 700, title = 'Sunburst chart'):
        """
        Displays a sunburst diagram of the results filtered by minimum number of matches. 
        """
        char, parent, match = self.mapIDs(minMatches)
        plotData = self.organizeData(char, parent, match)
        fig = px.sunburst(plotData,
                          names = 'LCA',
                          parents = 'Parent',
                          values = 'PercentMatches',
                          color = 'PercentMatches',
                          title = title,
                          width = width,
                          height = height,
        )

        fig.show()
           
    def showIcicle(self, minMatches = 5, width = 900, height = 2000, title = 'Icicle chart'):
        """
        Displays an icicle chart of the results filtered by minimum number of matches.
        """
        char, parent, match = self.mapIDs(minMatches)
        plotData = self.organizeData(char, parent, match)
        fig = px.icicle(plotData,
                        names ='LCA',
                        parents ='Parent',
                        values = 'PercentMatches',
                        color = 'PercentMatches',
                        title = title,
                        width = width,
                        height = height,
        )
        fig.update_traces(root_color="lightgrey")
        fig.update_layout(margin = dict(t=50, l=25, r=25, b=25)) # might need fine-tuning depending on machine
        fig.show()
    
    

#### Creek water

In [ ]:
creekWater = krakView('MYTEXT.kraken2')

In [ ]:
print(creekWater.showFormat(3))

In [ ]:
percentClassd = creekWater.percentClassified()

In [ ]:
nrOfUniques = creekWater.numberUniqueIDs()

In [ ]:
creekWater.showList()

In [ ]:
creekWater.showSunburst()

In [ ]:
creekWater.showIcicle()

In [ ]:
creekWater.showList()

#### Raw sewage


In [ ]:
sewage = krakView('sewageTXT.kraken2')

In [ ]:
sewage.showList(10)

In [ ]:
sewage.showSunburst(minMatches=1)

In [ ]:
sewage.showIcicle(minMatches = 1, height = 2800)

#### Fecal sample

In [ ]:
fecal = krakView('fecalTXT.kraken2')

In [ ]:
fecal.showList(35)

In [ ]:
fecal.showSunburst(minMatches=4)

In [ ]:
fecal.showIcicle(4, height=5000)

#### Endotracheal sample

In [ ]:
endotrac = krakView('endotrTXT.kraken2')

In [ ]:
endotrac.showList(minMatches=1)

In [ ]:
endotrac.showIcicle(minMatches=1)